In [1]:
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go
import plotly.express as px

In [4]:
import plotly.express as px
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import matplotlib.pyplot as plt
import geojsoncontour

from scipy.interpolate import griddata
from numpy import linspace

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
def performance_summary(model, X_test, y_test ):
    
    y_hat=model.predict(X_test)
    
    df_summary=pd.DataFrame(y_hat, columns=["y_hat"])
    df_summary["y_true"]=y_test
    df_summary["abs_error"]=np.abs(df_summary.y_true-df_summary.y_hat)
    df_summary["error"]=df_summary.y_hat-df_summary.y_true
    df_summary["relative_error"]= df_summary["error"]/df_summary.y_true
    df_summary["relative_abs_error"]= df_summary["abs_error"]/df_summary.y_true
    
    share_within_5pct=(df_summary.query("relative_abs_error<0.05").shape[0]/df_summary.shape[0])*100
    
    print("{:.2f}% : Share of forecasts within 5% absolute error\n".format(share_within_5pct))
    print("{:.0f}   : Mean absolute error \n".format(df_summary.abs_error.mean()))
    print("{:.2f}% : Mean absolute percentage error\n".format(df_summary.relative_abs_error.mean()*100))



In [7]:
def DNN_plot_loss(history, starting_epoch,previous_val_loss):

        trace0=go.Scatter(
                y=history.history['loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="blue",
                size=5,
                opacity=0.5
                ),
                name="Training Loss"
            )


        trace1=go.Scatter(
                y=history.history['val_loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="red",
                size=5,
                opacity=0.5
                ),
                name="Validation Loss"
            )
        
        trace2=go.Scatter(
                y=list(np.ones([len(history.epoch[starting_epoch:])])*np.asarray(previous_val_loss).min()),
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="grey",
                size=5,

                ),
                name="Lowest error from previous models"
            )

        data=[trace0, trace1,trace2]
        figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning curve",
                yaxis=dict(title="Loss",range=(900,1500)),
                xaxis=dict(title="Epoch",range=(starting_epoch,history.epoch[-1])),
                legend=dict(
                    x=0.57,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None,



            )))
        iplot(figure)

In [8]:
#Selecting a central city point to center all graphs around - Swietokrzyska Subway 
center_coors=52.235176, 21.008393

In [9]:
df = pd.read_csv(r"https://raw.githubusercontent.com/Jan-Majewski/Project_Portfolio/master/03_Real_Estate_pricing_in_Warsaw/top_features_data.csv")
df.shape

(24935, 46)

In [10]:
y=df.unit_price
X=df.drop(columns=["unit_price","lat_mod","lon_mod"])

## Spliting into train and test sets

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)


In [12]:
test_idx=np.asarray(X_test.index)

In [13]:
X_test_coors=df[[ "unit_price","lat_mod","lon_mod"]].iloc[test_idx]
X_test_coors.reset_index(inplace=True,drop=True)

In [14]:
df.columns

Index(['build_year', 'building_floors_num', 'rooms_num',
       'Equipment_types_dishwasher', 'Equipment_types_fridge',
       'Equipment_types_furniture', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_balcony', 'Extras_types_garden', 'Extras_types_lift',
       'floor_num', 'east_bank', 'distance_driving', 'distance_transit',
       'time_driving', 'time_transit', 'restaurant_price_level',
       'restaurant_mean_rating', 'restaurant_mean_popularity',
       'restaurant_count', 'restaurant_ratings_count', 'district_Bemowo',
       'district_Bialoleka', 'district_Downtown', 'district_Subburbs',
       'district_Targowek', 'district_Wawer', 'district_Wola',
       'district_Zoliborz', 'market_primary', 'Building_material_brick',
       'Building_ownership_full_ownership', 'Building_type_apartment',
       'Building_type_block', 'Building_type_tenement',
       'Construction_status_ready_to_use', 'Construction_status

In [15]:
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

# Neural networks models

## Transforming and scaling data for NN

In [16]:
y_train=np.asarray(y_train).reshape(-1,1)
y_train.shape

(19948, 1)

In [17]:
y_train.mean()

11195.887256867856

In [18]:
y_test=np.asarray(y_test).reshape(-1,1)
y_test.shape

(4987, 1)

In [19]:
y_test.mean()

11128.766392620813

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [22]:
X_train.shape

(19948, 43)

In [23]:
X_test.shape

(4987, 43)

## Initial model

In [24]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dense(units=256,activation='relu'),  
    keras.layers.Dense(units=256,activation='relu'),    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Initial_model",)


In [25]:
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 515,329
Trainable params: 515,329
Non-trainable params: 0
_______________________________________________

In [26]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 2s 120us/sample - loss: 11118.3712 - val_loss: 10586.6312
Epoch 2/200
19948/19948 [==============================] - 0s 16us/sample - loss: 7066.7532 - val_loss: 3763.7942
Epoch 3/200
19948/19948 [==============================] - 0s 11us/sample - loss: 2464.6214 - val_loss: 1684.2396
Epoch 4/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1628.4329 - val_loss: 1496.9290
Epoch 5/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1463.1655 - val_loss: 1407.6187
Epoch 6/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1387.9316 - val_loss: 1362.7425
Epoch 7/200
19948/19948 [==============================] - 0s 12us/sample - loss: 1347.6676 - val_loss: 1336.8407
Epoch 8/200
19948/19948 [==============================] - 0s 12us/sample - loss: 1319.9523 - val_loss: 1317.9880
Epoch 9/200
19948/19948 [=============

19948/19948 [==============================] - 0s 7us/sample - loss: 1047.9060 - val_loss: 1177.2465
Epoch 73/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1045.4430 - val_loss: 1186.7017
Epoch 74/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1040.1967 - val_loss: 1177.5787
Epoch 75/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1038.5179 - val_loss: 1178.2507
Epoch 76/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1030.5437 - val_loss: 1170.2154
Epoch 77/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1024.0594 - val_loss: 1168.6879
Epoch 78/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1018.7631 - val_loss: 1170.7921
Epoch 79/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1016.7170 - val_loss: 1177.4242
Epoch 80/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1015.7998 - val_los

19948/19948 [==============================] - 0s 7us/sample - loss: 796.0964 - val_loss: 1177.8279
Epoch 145/200
19948/19948 [==============================] - 0s 7us/sample - loss: 796.2166 - val_loss: 1181.7916
Epoch 146/200
19948/19948 [==============================] - 0s 7us/sample - loss: 789.8379 - val_loss: 1195.0252
Epoch 147/200
19948/19948 [==============================] - 0s 7us/sample - loss: 795.7245 - val_loss: 1179.7856
Epoch 148/200
19948/19948 [==============================] - 0s 7us/sample - loss: 806.3644 - val_loss: 1184.7226
Epoch 149/200
19948/19948 [==============================] - 0s 7us/sample - loss: 784.6511 - val_loss: 1175.4893
Epoch 150/200
19948/19948 [==============================] - 0s 7us/sample - loss: 780.3454 - val_loss: 1174.3319
Epoch 151/200
19948/19948 [==============================] - 0s 8us/sample - loss: 768.5436 - val_loss: 1196.8327
Epoch 152/200
19948/19948 [==============================] - 0s 9us/sample - loss: 786.0099 - val_loss

In [27]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=[0])

In [28]:
previous_val_loss=[history.history["val_loss"][-1]]
performance_summary(model, X_test, y_test )

39.02% : Share of forecasts within 5% absolute error

1201   : Mean absolute error 

10.32% : Mean absolute percentage error



## Adding Drop-out

In [29]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Dropout",)


In [30]:
model.summary()

Model: "Dropout"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               6579

In [31]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 1s 51us/sample - loss: 11104.9790 - val_loss: 10453.7489
Epoch 2/200
19948/19948 [==============================] - 0s 9us/sample - loss: 6566.8866 - val_loss: 2844.8539
Epoch 3/200
19948/19948 [==============================] - 0s 9us/sample - loss: 2426.2823 - val_loss: 1743.4345
Epoch 4/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1758.0013 - val_loss: 1499.9643
Epoch 5/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1648.7799 - val_loss: 1415.0016
Epoch 6/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1603.1399 - val_loss: 1372.3619
Epoch 7/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1572.5706 - val_loss: 1345.1741
Epoch 8/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1534.9148 - val_loss: 1323.9660
Epoch 9/200
19948/19948 [================

19948/19948 [==============================] - 0s 8us/sample - loss: 1345.5289 - val_loss: 1202.5442
Epoch 73/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1359.0984 - val_loss: 1214.5745
Epoch 74/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1360.6952 - val_loss: 1192.5583
Epoch 75/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1355.5503 - val_loss: 1196.9080
Epoch 76/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1367.3959 - val_loss: 1204.0640
Epoch 77/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1348.3049 - val_loss: 1212.7422
Epoch 78/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1356.7574 - val_loss: 1200.0463
Epoch 79/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1345.9441 - val_loss: 1199.1317
Epoch 80/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1360.1341 - val_lo

19948/19948 [==============================] - 0s 8us/sample - loss: 1319.1233 - val_loss: 1163.9970
Epoch 144/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1295.9626 - val_loss: 1161.6499
Epoch 145/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1305.3658 - val_loss: 1183.7885
Epoch 146/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1326.9102 - val_loss: 1198.1618
Epoch 147/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1304.7840 - val_loss: 1167.9697
Epoch 148/200
19948/19948 [==============================] - 0s 7us/sample - loss: 1307.4831 - val_loss: 1192.3912
Epoch 149/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1298.4878 - val_loss: 1168.7396
Epoch 150/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1310.5902 - val_loss: 1169.6830
Epoch 151/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1297.6384

In [32]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [33]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

38.32% : Share of forecasts within 5% absolute error

1153   : Mean absolute error 

9.87% : Mean absolute percentage error



## Adding batch normalization

In [34]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Batchnorm",)


In [35]:
model.summary()

Model: "Batchnorm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               13

In [36]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 2s 120us/sample - loss: 11193.2647 - val_loss: 11118.9994
Epoch 2/200
19948/19948 [==============================] - 0s 15us/sample - loss: 11176.2126 - val_loss: 11081.1899
Epoch 3/200
19948/19948 [==============================] - 0s 17us/sample - loss: 11140.8920 - val_loss: 11036.2633
Epoch 4/200
19948/19948 [==============================] - 0s 23us/sample - loss: 11086.9997 - val_loss: 10963.5562
Epoch 5/200
19948/19948 [==============================] - 0s 16us/sample - loss: 11010.1029 - val_loss: 10843.1285
Epoch 6/200
19948/19948 [==============================] - 0s 16us/sample - loss: 10903.1746 - val_loss: 10675.0468
Epoch 7/200
19948/19948 [==============================] - 0s 24us/sample - loss: 10760.0189 - val_loss: 10464.9079
Epoch 8/200
19948/19948 [==============================] - 0s 19us/sample - loss: 10573.2153 - val_loss: 10203.9690
Epoch 9/200
19948/1994

19948/19948 [==============================] - 0s 17us/sample - loss: 1189.7584 - val_loss: 1128.6164
Epoch 72/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1193.6923 - val_loss: 1120.0759
Epoch 73/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1175.0648 - val_loss: 1117.6392
Epoch 74/200
19948/19948 [==============================] - 0s 19us/sample - loss: 1202.2625 - val_loss: 1122.8787
Epoch 75/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1173.5467 - val_loss: 1118.2371
Epoch 76/200
19948/19948 [==============================] - 0s 23us/sample - loss: 1154.2019 - val_loss: 1108.5054
Epoch 77/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1158.4335 - val_loss: 1120.8856
Epoch 78/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1176.1779 - val_loss: 1138.6234
Epoch 79/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1160.9439 

Epoch 142/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1026.2975 - val_loss: 1107.8643
Epoch 143/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1013.4394 - val_loss: 1101.5852
Epoch 144/200
19948/19948 [==============================] - 0s 20us/sample - loss: 1009.9328 - val_loss: 1116.1110
Epoch 145/200
19948/19948 [==============================] - 0s 21us/sample - loss: 1018.7281 - val_loss: 1104.3793
Epoch 146/200
19948/19948 [==============================] - 0s 18us/sample - loss: 1009.7395 - val_loss: 1098.4590
Epoch 147/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1015.9798 - val_loss: 1095.2978
Epoch 148/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1002.6646 - val_loss: 1101.0200
Epoch 149/200
19948/19948 [==============================] - 1s 28us/sample - loss: 1086.8367 - val_loss: 1088.4710
Epoch 150/200
19948/19948 [==============================] - 0s 19us/sam

In [37]:
previous_val_loss[-1]

1152.6631266439226

In [38]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [39]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

43.25% : Share of forecasts within 5% absolute error

1092   : Mean absolute error 

9.42% : Mean absolute percentage error



## Adding Leaky Relu

In [40]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dense(units=1, activation="linear"),



],name="LeakyRELU",)


In [41]:
model.summary()

Model: "LeakyRELU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               20

In [42]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=300, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/300
19948/19948 [==============================] - 2s 102us/sample - loss: 11193.0127 - val_loss: 11115.5698
Epoch 2/300
19948/19948 [==============================] - 0s 22us/sample - loss: 11179.0485 - val_loss: 11076.8315
Epoch 3/300
19948/19948 [==============================] - 0s 15us/sample - loss: 11154.1231 - val_loss: 11051.6712
Epoch 4/300
19948/19948 [==============================] - 0s 14us/sample - loss: 11116.2460 - val_loss: 11000.3210
Epoch 5/300
19948/19948 [==============================] - 0s 15us/sample - loss: 11061.6303 - val_loss: 10916.0251
Epoch 6/300
19948/19948 [==============================] - 0s 14us/sample - loss: 10985.2781 - val_loss: 10823.6700
Epoch 7/300
19948/19948 [==============================] - 0s 21us/sample - loss: 10882.4478 - val_loss: 10676.5590
Epoch 8/300
19948/19948 [==============================] - 0s 17us/sample - loss: 10747.9906 - val_loss: 10486.9070
Epoch 9/300
19948/1994

19948/19948 [==============================] - 0s 14us/sample - loss: 1297.9863 - val_loss: 1191.5758
Epoch 72/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1298.5611 - val_loss: 1218.7372
Epoch 73/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1292.1154 - val_loss: 1181.7329
Epoch 74/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1287.8507 - val_loss: 1194.7818
Epoch 75/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1285.8532 - val_loss: 1176.6545
Epoch 76/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1282.9132 - val_loss: 1194.8673
Epoch 77/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1280.3733 - val_loss: 1190.3676
Epoch 78/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1276.3582 - val_loss: 1178.7731
Epoch 79/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1265.9267 

Epoch 142/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1222.1971 - val_loss: 1152.2460
Epoch 143/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1218.2522 - val_loss: 1175.2635
Epoch 144/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1233.3974 - val_loss: 1151.7260
Epoch 145/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1216.8632 - val_loss: 1141.8869
Epoch 146/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1214.8327 - val_loss: 1149.2506
Epoch 147/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1229.5001 - val_loss: 1154.5285
Epoch 148/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1220.1851 - val_loss: 1148.3292
Epoch 149/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1220.3347 - val_loss: 1147.5061
Epoch 150/300
19948/19948 [==============================] - 0s 16us/sam

19948/19948 [==============================] - 0s 13us/sample - loss: 1176.5006 - val_loss: 1124.2316
Epoch 213/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1178.5668 - val_loss: 1121.3620
Epoch 214/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1177.3751 - val_loss: 1139.5700
Epoch 215/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1175.1228 - val_loss: 1130.1917
Epoch 216/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1171.0865 - val_loss: 1128.8980
Epoch 217/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1173.1549 - val_loss: 1130.3500
Epoch 218/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1183.5341 - val_loss: 1118.5328
Epoch 219/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1171.2143 - val_loss: 1119.8638
Epoch 220/300
19948/19948 [==============================] - 0s 17us/sample - loss: 11

Epoch 283/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1143.6011 - val_loss: 1106.4081
Epoch 284/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1149.0149 - val_loss: 1113.8873
Epoch 285/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1138.9183 - val_loss: 1104.9291
Epoch 286/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1136.4113 - val_loss: 1107.5634
Epoch 287/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1142.2239 - val_loss: 1113.8885
Epoch 288/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1141.9110 - val_loss: 1107.5968
Epoch 289/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1149.4104 - val_loss: 1111.9556
Epoch 290/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1138.4921 - val_loss: 1104.6994
Epoch 291/300
19948/19948 [==============================] - 0s 16us/sam

In [43]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [44]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

41.33% : Share of forecasts within 5% absolute error

1099   : Mean absolute error 

9.37% : Mean absolute percentage error



## Creating a larger network

In [45]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.01),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Larger_network",)


In [46]:
model.summary()

Model: "Larger_network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              45056     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)            

In [47]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=300, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/300
19948/19948 [==============================] - 3s 140us/sample - loss: 11191.7813 - val_loss: 11104.0074
Epoch 2/300
19948/19948 [==============================] - 0s 21us/sample - loss: 11177.7166 - val_loss: 11047.1123
Epoch 3/300
19948/19948 [==============================] - 1s 27us/sample - loss: 11149.8176 - val_loss: 10989.5124
Epoch 4/300
19948/19948 [==============================] - 0s 23us/sample - loss: 11107.0977 - val_loss: 10920.8445
Epoch 5/300
19948/19948 [==============================] - 0s 18us/sample - loss: 11046.7887 - val_loss: 10824.1816
Epoch 6/300
19948/19948 [==============================] - 0s 19us/sample - loss: 10964.4873 - val_loss: 10680.5738
Epoch 7/300
19948/19948 [==============================] - 0s 18us/sample - loss: 10856.0065 - val_loss: 10554.1936
Epoch 8/300
19948/19948 [==============================] - 0s 22us/sample - loss: 10713.6386 - val_loss: 10368.5409
Epoch 9/300
19948/1994

19948/19948 [==============================] - 0s 16us/sample - loss: 1249.7765 - val_loss: 1171.7146
Epoch 72/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1251.0174 - val_loss: 1196.8244
Epoch 73/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1239.9494 - val_loss: 1172.2112
Epoch 74/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1245.8018 - val_loss: 1170.1588
Epoch 75/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1238.1144 - val_loss: 1161.0885
Epoch 76/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1238.1763 - val_loss: 1174.1115
Epoch 77/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1232.3501 - val_loss: 1154.4918
Epoch 78/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1234.3021 - val_loss: 1164.6497
Epoch 79/300
19948/19948 [==============================] - 1s 30us/sample - loss: 1233.7283 

Epoch 142/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1153.0398 - val_loss: 1147.5899
Epoch 143/300
19948/19948 [==============================] - 1s 28us/sample - loss: 1157.5704 - val_loss: 1138.6909
Epoch 144/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1154.6896 - val_loss: 1133.3848
Epoch 145/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1153.9761 - val_loss: 1133.5609
Epoch 146/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1158.1799 - val_loss: 1145.1097
Epoch 147/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1156.3617 - val_loss: 1138.7427
Epoch 148/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1149.2112 - val_loss: 1138.0037
Epoch 149/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1154.7119 - val_loss: 1130.6011
Epoch 150/300
19948/19948 [==============================] - 0s 17us/sam

19948/19948 [==============================] - 0s 18us/sample - loss: 1110.6295 - val_loss: 1110.5566
Epoch 213/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1103.3080 - val_loss: 1107.1132
Epoch 214/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1093.1509 - val_loss: 1112.7981
Epoch 215/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1109.7088 - val_loss: 1111.0712
Epoch 216/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1101.7280 - val_loss: 1109.9361
Epoch 217/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1102.9613 - val_loss: 1108.9369
Epoch 218/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1101.7768 - val_loss: 1115.7609
Epoch 219/300
19948/19948 [==============================] - 0s 22us/sample - loss: 1103.6902 - val_loss: 1105.0741
Epoch 220/300
19948/19948 [==============================] - 0s 17us/sample - loss: 10

Epoch 283/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1068.2543 - val_loss: 1094.8821
Epoch 284/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1064.3840 - val_loss: 1103.7309
Epoch 285/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1060.6612 - val_loss: 1095.4342
Epoch 286/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1066.0253 - val_loss: 1098.9363
Epoch 287/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1068.9224 - val_loss: 1101.9530
Epoch 288/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1070.9516 - val_loss: 1098.0746
Epoch 289/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1063.8086 - val_loss: 1097.1329
Epoch 290/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1065.2417 - val_loss: 1091.4558
Epoch 291/300
19948/19948 [==============================] - 0s 21us/sam

In [48]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [49]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

41.53% : Share of forecasts within 5% absolute error

1097   : Mean absolute error 

9.50% : Mean absolute percentage error



## Adjusting learning rate decay

In [50]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.01),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Learning_rate_decay",)


In [51]:
learning_rate = 0.005
decay = 5e-4
n_epochs=400
n_steps_per_epoch = len(X_train) // 1024
epochs = np.arange(n_epochs)
lrs = learning_rate / (1 + decay * epochs * n_steps_per_epoch)

In [52]:


trace0=go.Scatter(
            y=lrs,
            x=epochs,
            mode='lines',
            marker=dict(
            color="red",
            size=5,
            opacity=0.5
            )
    )
        

data=[trace0]
figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning rate decay",
                yaxis=dict(title="Learning rate"),
                xaxis=dict(title="Epoch"),
                legend=dict(
                    x=1,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None


            )))
iplot(figure)

In [53]:
optimizer = keras.optimizers.Adam(lr=0.005, decay=5e-4)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=400, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/400
19948/19948 [==============================] - 3s 134us/sample - loss: 11171.3894 - val_loss: 10215.8105
Epoch 2/400
19948/19948 [==============================] - 0s 17us/sample - loss: 10988.3311 - val_loss: 8093.7663
Epoch 3/400
19948/19948 [==============================] - 0s 19us/sample - loss: 10369.8484 - val_loss: 6920.4378
Epoch 4/400
19948/19948 [==============================] - 0s 18us/sample - loss: 8943.6729 - val_loss: 8042.0203
Epoch 5/400
19948/19948 [==============================] - 0s 17us/sample - loss: 6391.3162 - val_loss: 15662.6513
Epoch 6/400
19948/19948 [==============================] - 0s 20us/sample - loss: 4040.4297 - val_loss: 10329.9911
Epoch 7/400
19948/19948 [==============================] - 0s 25us/sample - loss: 1864.3942 - val_loss: 12797.6682
Epoch 8/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1578.0143 - val_loss: 8774.8852
Epoch 9/400
19948/19948 [======

Epoch 72/400
19948/19948 [==============================] - 0s 24us/sample - loss: 1208.1765 - val_loss: 1169.4947
Epoch 73/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1187.2578 - val_loss: 1150.5507
Epoch 74/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1189.0278 - val_loss: 1141.2771
Epoch 75/400
19948/19948 [==============================] - 0s 24us/sample - loss: 1192.7991 - val_loss: 1161.2935
Epoch 76/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1187.8521 - val_loss: 1164.6987
Epoch 77/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1183.7498 - val_loss: 1147.6664
Epoch 78/400
19948/19948 [==============================] - 1s 28us/sample - loss: 1189.7391 - val_loss: 1147.7745
Epoch 79/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1191.2774 - val_loss: 1166.7738
Epoch 80/400
19948/19948 [==============================] - 0s 19us/sample - los

Epoch 143/400
19948/19948 [==============================] - 0s 25us/sample - loss: 1119.0780 - val_loss: 1110.7133
Epoch 144/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1125.3924 - val_loss: 1113.9108
Epoch 145/400
19948/19948 [==============================] - 1s 29us/sample - loss: 1118.2526 - val_loss: 1117.8953
Epoch 146/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1118.0542 - val_loss: 1113.4458
Epoch 147/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1118.5877 - val_loss: 1116.5380
Epoch 148/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1103.0027 - val_loss: 1111.1589
Epoch 149/400
19948/19948 [==============================] - 1s 28us/sample - loss: 1122.8149 - val_loss: 1105.3445
Epoch 150/400
19948/19948 [==============================] - 1s 25us/sample - loss: 1121.8092 - val_loss: 1108.5650
Epoch 151/400
19948/19948 [==============================] - 0s 20us/sam

19948/19948 [==============================] - 0s 19us/sample - loss: 1074.3685 - val_loss: 1092.9811
Epoch 214/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1068.7884 - val_loss: 1091.0727
Epoch 215/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1079.5693 - val_loss: 1092.2428
Epoch 216/400
19948/19948 [==============================] - 1s 27us/sample - loss: 1069.8279 - val_loss: 1093.1142
Epoch 217/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1073.9470 - val_loss: 1094.4866
Epoch 218/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1072.6508 - val_loss: 1092.9297
Epoch 219/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1082.4189 - val_loss: 1094.5677
Epoch 220/400
19948/19948 [==============================] - 1s 30us/sample - loss: 1070.8349 - val_loss: 1096.0116
Epoch 221/400
19948/19948 [==============================] - 0s 20us/sample - loss: 10

Epoch 284/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1048.2490 - val_loss: 1087.2652
Epoch 285/400
19948/19948 [==============================] - 1s 25us/sample - loss: 1051.5229 - val_loss: 1080.3830
Epoch 286/400
19948/19948 [==============================] - 1s 28us/sample - loss: 1046.6626 - val_loss: 1088.4076
Epoch 287/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1057.5062 - val_loss: 1080.8944
Epoch 288/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1053.9465 - val_loss: 1081.9508
Epoch 289/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1054.5748 - val_loss: 1082.4989
Epoch 290/400
19948/19948 [==============================] - 1s 27us/sample - loss: 1047.0767 - val_loss: 1085.7088
Epoch 291/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1051.9027 - val_loss: 1083.0210
Epoch 292/400
19948/19948 [==============================] - 0s 20us/sam

19948/19948 [==============================] - 0s 23us/sample - loss: 1035.8678 - val_loss: 1076.9038
Epoch 355/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1029.5436 - val_loss: 1077.1456
Epoch 356/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1038.0853 - val_loss: 1074.7141
Epoch 357/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1026.0475 - val_loss: 1074.8276
Epoch 358/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1025.2332 - val_loss: 1074.7612
Epoch 359/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1033.8520 - val_loss: 1072.3684
Epoch 360/400
19948/19948 [==============================] - 1s 36us/sample - loss: 1016.8170 - val_loss: 1070.7449
Epoch 361/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1028.4912 - val_loss: 1077.1841
Epoch 362/400
19948/19948 [==============================] - 0s 22us/sample - loss: 10

In [54]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [55]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

42.31% : Share of forecasts within 5% absolute error

1075   : Mean absolute error 

9.25% : Mean absolute percentage error



## Callbacks stop

In [56]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.01),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Callbacks_stop",)


In [57]:
checkpoint_name = 'Weights\Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [58]:
optimizer = keras.optimizers.Adam(lr=0.005, decay=5e-4)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=500, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    callbacks=callbacks_list, 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/500
17408/19948 [=========================>....] - ETA: 0s - loss: 11178.8039
Epoch 00001: val_loss improved from inf to 10212.57571, saving model to Weights\Weights-001--10212.57571.hdf5
19948/19948 [==============================] - 3s 154us/sample - loss: 11171.3879 - val_loss: 10212.5757
Epoch 2/500
17408/19948 [=========================>....] - ETA: 0s - loss: 11019.9533
Epoch 00002: val_loss improved from 10212.57571 to 7787.54832, saving model to Weights\Weights-002--7787.54832.hdf5
19948/19948 [==============================] - 1s 28us/sample - loss: 10988.2913 - val_loss: 7787.5483
Epoch 3/500
19456/19948 [============================>.] - ETA: 0s - loss: 10378.0419
Epoch 00003: val_loss improved from 7787.54832 to 6571.05155, saving model to Weights\Weights-003--6571.05155.hdf5
19948/19948 [==============================] - 1s 36us/sample - loss: 10369.8656 - val_loss: 6571.0515
Epoch 4/500
19456/19948 [================

Epoch 30/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1302.3485
Epoch 00030: val_loss improved from 1222.08511 to 1220.06825, saving model to Weights\Weights-030--1220.06825.hdf5
19948/19948 [==============================] - 1s 35us/sample - loss: 1301.9109 - val_loss: 1220.0683
Epoch 31/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1291.5311
Epoch 00031: val_loss improved from 1220.06825 to 1211.83694, saving model to Weights\Weights-031--1211.83694.hdf5
19948/19948 [==============================] - 1s 30us/sample - loss: 1289.2417 - val_loss: 1211.8369
Epoch 32/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1284.3923
Epoch 00032: val_loss improved from 1211.83694 to 1205.20489, saving model to Weights\Weights-032--1205.20489.hdf5
19948/19948 [==============================] - 1s 31us/sample - loss: 1283.5300 - val_loss: 1205.2049
Epoch 33/500
19456/19948 [============================>.] - ETA: 0s - loss: 1281.3039
Epoch

Epoch 61/500
19456/19948 [============================>.] - ETA: 0s - loss: 1206.7843
Epoch 00061: val_loss did not improve from 1163.29106
19948/19948 [==============================] - 0s 17us/sample - loss: 1210.7726 - val_loss: 1172.5246
Epoch 62/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1208.3193
Epoch 00062: val_loss did not improve from 1163.29106
19948/19948 [==============================] - 0s 16us/sample - loss: 1212.2413 - val_loss: 1163.2916
Epoch 63/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1215.4892
Epoch 00063: val_loss did not improve from 1163.29106
19948/19948 [==============================] - 0s 18us/sample - loss: 1212.0075 - val_loss: 1169.0116
Epoch 64/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1208.5356
Epoch 00064: val_loss did not improve from 1163.29106
19948/19948 [==============================] - 0s 19us/sample - loss: 1208.2433 - val_loss: 1167.0751
Epoch 65/500
17408/19948 [======

Epoch 93/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1170.5467
Epoch 00093: val_loss did not improve from 1136.37474
19948/19948 [==============================] - 0s 22us/sample - loss: 1168.3901 - val_loss: 1149.7063
Epoch 94/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1157.9013
Epoch 00094: val_loss did not improve from 1136.37474
19948/19948 [==============================] - 0s 23us/sample - loss: 1160.2907 - val_loss: 1142.1767
Epoch 95/500
19456/19948 [============================>.] - ETA: 0s - loss: 1161.8297
Epoch 00095: val_loss did not improve from 1136.37474
19948/19948 [==============================] - 0s 19us/sample - loss: 1162.1122 - val_loss: 1146.4462
Epoch 96/500
19456/19948 [============================>.] - ETA: 0s - loss: 1158.6322
Epoch 00096: val_loss did not improve from 1136.37474
19948/19948 [==============================] - 0s 21us/sample - loss: 1157.2077 - val_loss: 1142.1643
Epoch 97/500
17408/19948 [======

Epoch 125/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1136.2434
Epoch 00125: val_loss improved from 1118.39590 to 1117.43091, saving model to Weights\Weights-125--1117.43091.hdf5
19948/19948 [==============================] - 1s 28us/sample - loss: 1132.7100 - val_loss: 1117.4309
Epoch 126/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1126.6963
Epoch 00126: val_loss did not improve from 1117.43091
19948/19948 [==============================] - 0s 22us/sample - loss: 1127.1259 - val_loss: 1122.8788
Epoch 127/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1130.7035
Epoch 00127: val_loss did not improve from 1117.43091
19948/19948 [==============================] - 0s 20us/sample - loss: 1127.8761 - val_loss: 1126.8128
Epoch 128/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1131.0003
Epoch 00128: val_loss did not improve from 1117.43091
19948/19948 [==============================] - 0s 19us/sample - loss:

...

Epoch 355/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1024.5324
Epoch 00355: val_loss did not improve from 1070.09350
19948/19948 [==============================] - 0s 17us/sample - loss: 1024.6986 - val_loss: 1073.2758
Epoch 356/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1018.7953
Epoch 00356: val_loss did not improve from 1070.09350
19948/19948 [==============================] - 0s 17us/sample - loss: 1019.6574 - val_loss: 1074.6922
Epoch 357/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1021.5461
Epoch 00357: val_loss did not improve from 1070.09350
19948/19948 [==============================] - 0s 19us/sample - loss: 1019.6290 - val_loss: 1073.0519
Epoch 358/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1015.7233
Epoch 00358: val_loss did not improve from 1070.09350
19948/19948 [==============================] - 0s 17us/sample - loss: 1012.1404 - val_loss: 1070.9593
Epoch 359/500
18432/19948 [=

19456/19948 [============================>.] - ETA: 0s - loss: 1015.4311
Epoch 00388: val_loss did not improve from 1067.17286
19948/19948 [==============================] - 0s 19us/sample - loss: 1015.9287 - val_loss: 1076.2110
Epoch 389/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1011.4330
Epoch 00389: val_loss did not improve from 1067.17286
19948/19948 [==============================] - 0s 19us/sample - loss: 1012.9957 - val_loss: 1068.0222
Epoch 390/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1009.6128
Epoch 00390: val_loss did not improve from 1067.17286
19948/19948 [==============================] - 0s 16us/sample - loss: 1010.0891 - val_loss: 1069.4872
Epoch 391/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1009.8359
Epoch 00391: val_loss did not improve from 1067.17286
19948/19948 [==============================] - 0s 17us/sample - loss: 1009.7798 - val_loss: 1077.3770
Epoch 392/500
19456/19948 [===============

Epoch 421/500
19456/19948 [============================>.] - ETA: 0s - loss: 1016.8432
Epoch 00421: val_loss did not improve from 1065.30508
19948/19948 [==============================] - 1s 26us/sample - loss: 1015.1630 - val_loss: 1074.5026
Epoch 422/500
19456/19948 [============================>.] - ETA: 0s - loss: 1006.3546
Epoch 00422: val_loss did not improve from 1065.30508
19948/19948 [==============================] - 0s 21us/sample - loss: 1007.7487 - val_loss: 1070.2727
Epoch 423/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1007.6329
Epoch 00423: val_loss did not improve from 1065.30508
19948/19948 [==============================] - 0s 19us/sample - loss: 1005.8261 - val_loss: 1073.8129
Epoch 424/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1006.9275
Epoch 00424: val_loss did not improve from 1065.30508
19948/19948 [==============================] - 0s 20us/sample - loss: 1005.8377 - val_loss: 1070.2098
Epoch 425/500
19456/19948 [=

Epoch 455/500
19456/19948 [============================>.] - ETA: 0s - loss: 997.4192
Epoch 00455: val_loss did not improve from 1064.89246
19948/19948 [==============================] - 0s 20us/sample - loss: 999.9419 - val_loss: 1071.9366
Epoch 456/500
19456/19948 [============================>.] - ETA: 0s - loss: 995.4572
Epoch 00456: val_loss did not improve from 1064.89246
19948/19948 [==============================] - 0s 24us/sample - loss: 995.3234 - val_loss: 1070.9992
Epoch 457/500
19456/19948 [============================>.] - ETA: 0s - loss: 990.2454
Epoch 00457: val_loss did not improve from 1064.89246
19948/19948 [==============================] - 0s 25us/sample - loss: 990.1067 - val_loss: 1065.0477
Epoch 458/500
19456/19948 [============================>.] - ETA: 0s - loss: 1000.0172
Epoch 00458: val_loss improved from 1064.89246 to 1064.46231, saving model to Weights\Weights-458--1064.46231.hdf5
19948/19948 [==============================] - 1s 26us/sample - loss: 1000.

Epoch 488/500
17408/19948 [=========================>....] - ETA: 0s - loss: 982.3995
Epoch 00488: val_loss did not improve from 1062.22118
19948/19948 [==============================] - 1s 42us/sample - loss: 985.4887 - val_loss: 1067.6095
Epoch 489/500
19456/19948 [============================>.] - ETA: 0s - loss: 982.3693
Epoch 00489: val_loss did not improve from 1062.22118
19948/19948 [==============================] - 1s 30us/sample - loss: 982.6062 - val_loss: 1066.7865
Epoch 490/500
19456/19948 [============================>.] - ETA: 0s - loss: 992.2404
Epoch 00490: val_loss did not improve from 1062.22118
19948/19948 [==============================] - 1s 36us/sample - loss: 990.2624 - val_loss: 1067.5991
Epoch 491/500
19456/19948 [============================>.] - ETA: 0s - loss: 989.4027
Epoch 00491: val_loss did not improve from 1062.22118
19948/19948 [==============================] - 1s 30us/sample - loss: 988.8071 - val_loss: 1067.2337
Epoch 492/500
19456/19948 [=========

In [59]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [67]:
import glob
import os

list_of_files = glob.glob('Weights/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getmtime)

In [68]:
weights_file =  latest_file # choose the best checkpoint 
model.load_weights(weights_file) # load it
model.compile(loss='mean_absolute_error', optimizer='adam')

In [81]:
previous_val_loss.append(np.asarray(history.history["val_loss"]).min())
performance_summary(model, X_test, y_test )

43.33% : Share of forecasts within 5% absolute error

1062   : Mean absolute error 

9.12% : Mean absolute percentage error



In [82]:
previous_val_loss

[1200.5697104463793,
 1152.6631266439226,
 1092.2156825275013,
 1099.1921439590592,
 1097.4319312732637,
 1074.8323216045312,
 1062.2211774050031]

In [83]:
models=["basic_model","dropout","batch_norm","leakyRELU","1024_layer","lr_decay","callbacks"]

In [84]:
df_comparison=pd.DataFrame(previous_val_loss,columns=["val_loss"])
df_comparison["model"]=models

In [85]:
import plotly.express as px

fig = px.line(df_comparison, x="model", y="val_loss")
fig.show()